In [0]:
dbutils.widgets.text("catalog", "amitabh_arora_catalog")
dbutils.widgets.text("schema", "kaggle_world_wide_importers")
schema_path = dbutils.widgets.get("schema_path")

In [0]:
tables_df = spark.sql(f"SHOW TABLES IN {catalog}.{schema}")
tables_df = tables_df.filter(~tables_df.tableName.startswith('_'))
display(tables_df)

In [0]:
spark.sql(f"USE CATALOG {catalog}")

In [0]:
%sql
select table_name, constraint_name, constraint_type from information_schema.table_constraints

In [0]:
%sql
select table_name, constraint_name, column_name  from information_schema.constraint_column_usage


In [0]:
df = spark.sql(f"""
  select
    tc.table_name, ccu.column_name, tc.constraint_name, tc.constraint_type,
    null as foreign_table_name, null as foreign_column_name
  from information_schema.constraint_column_usage as ccu
  left outer join information_schema.table_constraints as tc
    -- on ccu.table_name = tc.table_name
    on ccu.constraint_name = tc.constraint_name
  where ccu.table_schema = '{schema}'
  and tc.constraint_type = 'PRIMARY KEY'
  union all
  select
    tc.table_name, ccu.column_name, tc.constraint_name, tc.constraint_type,
    ccu.table_name as foreign_table_name, ccu.column_name as foreign_column_name
    from information_schema.table_constraints as tc
  left outer join information_schema.constraint_column_usage as ccu
    -- on tc.table_name = ccu.table_name
    on tc.constraint_name = ccu.constraint_name
  where tc.constraint_type = 'FOREIGN KEY'
  and tc.table_schema = '{schema}'
""")

display(df)